# Creating an EPFL Courses Graph

In the notebook similarity_method, we established and evaluated the method for calculating similarity/relatedness between two EPFL courses. In this notebook, __we will create a graph of EPFL courses.__ We want to create a graph in such a way that two related courses share a link. 

The notebook is divided into two parts:
1. __Pre-processing:__ Data cleaning, fixing inconsistencies, pre-processing pipeline, creating final dataframe. 
2. __Graph-building:__ Using similarity method on pre-processed course data to build the graph 


In [1]:
%matplotlib notebook

import nltk, re, itertools, getpass, pymysql.cursors
import pandas as pd
import numpy as np
import string

#sklearn
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

#gensim
import gensim
from gensim.models import Doc2Vec
from gensim.models import KeyedVectors
from gensim.models.doc2vec import TaggedDocument

#nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /Users/rinjac/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/rinjac/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

## 1. Pre-processing

### 1.1 Data cleaning

Pre-processing is the similar as in the notebook similarity_method.

Firstly, let's get the datasets. We have three datasets, which are:
1. __course_descriptions__ - Contains course name, description, summary, and other data for __all EPFL courses__
2. __course_keywords__ - Contains keywords for each EPFL course (Keywords are usually main concepts taught in the course)
3. __course_semesters__ - Contains in which semester is each course mostly taken (for example, Master 2. semester)

In [2]:
df = pd.read_csv('datasets/course_desc.csv', index_col=0, keep_default_na=False)
kwdf = pd.read_csv('datasets/course_keywords.csv', index_col=0, keep_default_na=False)
smdf = pd.read_csv('datasets/course_semesters.csv', index_col=0, keep_default_na=False)

Fixing minor inconsistencies in the datasets.

In [3]:
# in a lof of database rows, we don't have the course title in English ('CourseNameEN')
# but the 'CourseNameFR' field is actually title in English
# so we fix this
df['CourseName'] = df.apply(lambda row: row['CourseNameFR'] if not row['CourseNameEN'] else row['CourseNameEN'], axis=1)
df = df.drop(columns=['CourseNameEN', 'CourseNameFR'])
df['CourseName'] = df['CourseName'].apply(lambda x: " " if not x else x)

# concat the course content and summary
df['CourseContent'] = df['CourseContent'] + df['SummaryEN']
df = df.drop(columns=['SummaryEN'])

# remove rows which still have null values
df = df.dropna()

Adding keywords to main dataframe.

In [4]:
# group keywords by course code
kwdf = kwdf.groupby('CourseCode')['TagValue'].agg(lambda col: ' '.join(col))
kwdf = pd.DataFrame(kwdf)

# add keywords for DF dataframe
df = df.set_index(keys=['CourseCode'], drop=True)
df = df.merge(kwdf, on=['CourseCode'])
df = df.reset_index()

Defining pre-processing functions we are going to use:

In [5]:
def remove_nonascii(s):
    return s.encode('ascii', 'ignore').decode("utf-8")

def remove_newline(s):
    return s.replace('\n','')

def remove_squote(s):
    return s.replace('<squote/>',' ')

stop_words = stopwords.words('english')
def remove_stop_words(tokens):
    return [word for word in tokens if word not in stop_words]

french_stop_words = stopwords.words('french')
def remove_french_stop_words(tokens):
    return [word for word in tokens if word not in french_stop_words]

punc = list(string.punctuation)
def remove_punc(tokens):
    return [word for word in tokens if word not in punc]

def to_lower(tokens):
    return [token.lower() for token in tokens]

def apply_preproc(df, column, func): 
    df[column] = df[column].apply(func)

### 1.2 Finding most common words

__For reasons explained in notebook similarity_method (in the section 2.2)__ we will write the code which finds the N most common words in course descriptions and a pre-processing function which will remove them. 

In [6]:
ndf = df.copy()

str_list = []
for i in range(len(df)):
    str_list.append(df.iloc[i]['CourseContent'])
all_content = ''.join(str_list)
                    
all_content = word_tokenize(all_content)
all_content = to_lower(all_content)
all_content = remove_stop_words(all_content)
all_content = remove_punc(all_content)

freq_d = dict()
for w in all_content:
    freq_d[w] = 1 + freq_d.get(w, 0)
    
freq = [(freq_d[key], key) for key in freq_d]
freq.sort()
freq.reverse()

most_common_f = freq[:100]
_, most_common = [list(tup) for tup in zip(*most_common_f)]
print(most_common)

['course', 'semicolon/', 'students', 'design', 'systems', 'analysis', 'methods', 'introduction', 'squote/', 'basic', 'models', 'techniques', 'energy', 'project', 'theory', 'concepts', '2', 'applications', 'materials', 'data', 'research', '3', 'different', 'processes', 'principles', 'system', 'tools', 'student', 'development', 'work', 'well', 'engineering', 'field', 'linear', 'topics', 'management', 'properties', 'part', 'also', 'control', '4', 'use', 'main', 'understanding', 'practical', 'learning', 'social', 'knowledge', 'using', 'study', 'understand', 'modeling', 'theoretical', 'problems', 'examples', 'various', 'structure', 'aspects', 'digital', 'science', '5', 'process', 'urban', 'fundamental', 'based', '1', 'application', 'new', "''", 'specific', 'used', 'processing', 'issues', 'information', 'time', 'approach', 'power', 'model', 'structures', 'learn', 'physical', 'chemical', '``', 'studies', 'case', 'technology', 'related', 'general', 'types', 'optical', 'quantum', 'physics', 'in

In [7]:
def remove_most_common_words(tokens):
    return [word for word in tokens if word not in most_common]

### 1.3 Creating final dataframe

We are now going to use pre-processing functions to create the final dataframe.

In [8]:
def apply_preproc(df, column, func): 
    df[column] = df[column].apply(func)
    
def keep_unique_courses(df):
    df['CourseCode'] = df['CourseCode'].apply(lambda x: x.split('(')[0])
    df = df.drop_duplicates(subset=['CourseCode'], keep='first')
    return df

# we need course name for graph visualizaion
df['CourseName2'] = df['CourseName']

In [9]:
# keep unique courses (some courses have variants (a), (b), (c))
df = keep_unique_courses(df)

columns = ['CourseContent', 'CourseName']

for column in columns:
    # tokenization
    apply_preproc(df, column, word_tokenize)
    # to lower
    apply_preproc(df, column, to_lower)
    # remove stop words
    apply_preproc(df, column, remove_stop_words)
    # remove french stop words
    apply_preproc(df, column, remove_french_stop_words)
    # remove punc
    apply_preproc(df, column, remove_punc)

# remove most common words
# ONLY for the course content
apply_preproc(df, 'CourseContent', remove_most_common_words)

# merge name and content
df['CourseContent'] = df['CourseName'] + df['CourseContent']
df = df[['CourseCode', 'CourseContent', 'CourseName2']]

We also need to pre-process the semesters dataframe.

In [10]:
smdf = smdf.sort_values('NRegistrations', ascending=False).drop_duplicates(['CourseCode'])
smdf = smdf[['CourseCode', 'PedagogicalName']]
smdf = smdf.dropna()

# keep unique courses (some courses have variants (a), (b), (c))
smdf = keep_unique_courses(smdf)

smdf = smdf.set_index(['CourseCode'])

Converting the semester names into numerical values.

In [11]:
def convert_pedname(pedname):
    if pedname == 'Bachelor semestre 1':
        return 1
    elif pedname == 'Bachelor semestre 2':
        return 2
    elif pedname == 'Bachelor semestre 3':
        return 3
    elif pedname == 'Bachelor semestre 4':
        return 4
    elif pedname == 'Bachelor semestre 5':
        return 5
    elif pedname == 'Bachelor semestre 5b':
        return 5
    elif pedname == 'Bachelor semestre 6':
        return 6
    elif pedname == 'Bachelor semestre 6b':
        return 6
    elif pedname == 'Master semestre 1':
        return 7
    elif pedname == 'Master semestre 2':
        return 8
    elif pedname == 'Master semestre 3':
        return 9
    elif pedname == 'Master semestre 4':
        return 10
    elif pedname == 'Projet master automne':
        return 9
    elif pedname == 'Projet master printemps':
        return 10
    elif pedname == 'Semestre automne':
        return 9
    elif pedname == 'Semestre printemps':
        return 10

In [12]:
smdf['PedagogicalName'] = smdf['PedagogicalName'].apply(lambda x: convert_pedname(x))
sem_dict = smdf.to_dict('index')

## 2. Graph-building

In this section we will __build the final graph of EPFL courses.__ 

The graph will have the following structure:
* __Nodes:__ EPFL courses
* __Edges:__ Two courses will share an edge if they are related (according to our similarity method)

The graph is going to be __directed__. We have the information which courses are taken when (course_semesters dataset), and we will __only conect courses from earlier to later.__


In [13]:
# pretrained word2vec on wikipedia
model = KeyedVectors.load_word2vec_format('wiki-news-300d-1M.vec')
EMBEDDING_SIZE = 300

In [14]:
#function for getting the average word embedding out of list of words
def get_average_vector(words_list):
    base = np.zeros(EMBEDDING_SIZE)
    word_vec = 0
    n = 0
    for word in words_list:
        try:
            word_vec = model[word]
            n += 1
        except KeyError:
            word_vec = np.zeros(300)
        base = np.add(base, word_vec)
    base = np.divide(base, n)
    
    return base.reshape(1, -1)

Selecting a threshold. Based on experiences in similarity_method, we will pick a threshold of 0.88.

In [15]:
THRESHOLD = 0.88

Building the graph.

In [18]:
# node
subjectID = []
subject_name = []
course_codes = []

# edge
source = []
target = []
type_ = []

for i in range(len(df)):
    list1 = df.iloc[i]['CourseContent']
    base1 = get_average_vector(list1)
    code1 = df.iloc[i]['CourseCode']
    
    # make node
    subjectID.append(str(i))
    subject_name.append(df.iloc[i]['CourseName2'])
    course_codes.append(code1)
    
    nodes = []
    for j in range(i+1, len(df)):
        code2 = df.iloc[j]['CourseCode']
        list2 = df.iloc[j]['CourseContent']
        base2 = get_average_vector(list2)

        sim = cosine_similarity(base1, base2)
        sim = sim[0][0]
        
        if sim < THRESHOLD:
            continue
        
        # as said, we only connect from earlier course to latter
        if sem_dict.get(code1) is not None and sem_dict.get(code2) is not None:
            if sem_dict.get(code1)['PedagogicalName'] < sem_dict.get(code2)['PedagogicalName']:
                nodes.append((j, sim, 'sd'))
            elif sem_dict.get(code1)['PedagogicalName'] > sem_dict.get(code2)['PedagogicalName']:
                nodes.append((j, sim, 'ds'))
    
    # we take top 3 most relevant courses for this course, and connect them to it
    nodes = sorted(nodes, key=lambda x: x[1])
    nodes = nodes[:3]
    for node in nodes:
        j = node[0]
        
        if node[2] == 'sd':
            source.append(str(i))
            target.append(str(j))
        else:
            source.append(str(j))
            target.append(str(i))
        type_.append("indicative")

Saving the graph's nodes and edges.

In [20]:
d = {'source': source, 'target': target, 'type': type_}
edges = pd.DataFrame(data=d)
edges.to_csv('graph/edges.csv', index=False)

d = {'id': subjectID, 'subjectname': subject_name, 'associatedcoursecodes': course_codes}
nodes = pd.DataFrame(data=d)
nodes.to_csv('graph/nodes.csv', index=False)